In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from shutil import copy

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

1) loading data and collecting it nice tide array (X and y)
2) making the keras model, like for mnist 
3) testing it and making graphs
4) writing an abstract

In [2]:
DATADIR = "C:/Users/verat/Desktop/DS_MeetUp/SpeechRecognition/specs08"
CATEGORIES = list(range(0,12+1))
#path = os.path.join(DATADIR,category)

In [3]:
#puts files with numbers in number-named folders
def folder_putter(path, Num):
    Num = str(Num)
    #check is /on the end of path 
    if (path[:-2]!='/'): path = path+'/'
    #check is folder exists, if not - create
    if (os.path.isdir(path+Num)==0):
        os.mkdir(path+Num)
    #find files starting that number and copy it in their folder
    filelist = os.listdir(path)
    for file in filelist:
        if (separ_number_from_filename(file)==Num) and os.path.isfile(path+Num+'/'+file)==0 and file.endswith(".png"):
            copy(path+file, path+Num+'/'+file)
    return 0


def separ_number_from_filename(filename_string):
    pos = len(filename_string)
    for i in range(len(filename_string)):
        if not filename_string[i].isdigit():
            pos = i
            break
    return str(int(filename_string[:pos])) 


In [4]:
#Put png file by they number categories
for Num in CATEGORIES:
    folder_putter(DATADIR, Num)

In [5]:
training_data = []
IMG_SIZE = 72

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,str(category))  # create path to each spoken number spectrogramm
        class_num = category  # get the classification  ( from 0 to 12)

        for img in tqdm(os.listdir(path)):  # iterate over each image 
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                #new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([img_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            
create_training_data()

print(len(training_data))

100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 438.88it/s]


716


In [18]:
import random
random.shuffle(training_data)
#split here


In [24]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

#X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
#X = np.array(X)/255.0

len(X)
X = np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 1)
X.shape

(17184, 72, 72, 1)

Next part - building the model

In [59]:
len(y)

716

In [60]:
X.shape

(17184, 72, 72, 1)

In [64]:
training_data[0]

[array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), 6]

this code below doesn't need any previous, because it takes images from directory, opens and resize them with *ImageDataGenerator*
But it does need files in the "categofy_name" folders

Need a splitter function for: getting the list of files, splitting them, putting files in "training" and "validation" folders, and  "categofy_name" folders 

In [32]:
#function for: 
#getting the list of files, 
#splitting them, 
#putting files in "training" and "validation" folders, and "categofy_name" folders 

def splitter ()

In [26]:
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "specs08/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=0,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(72,72),
	class_mode='categorical'
)


Found 716 images belonging to 13 classes.


In [29]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(72, 72, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(13, activation='softmax')
])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 70, 70, 64)        1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 33, 33, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 5, 128)        

In [30]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) #typical  optimizer='adam'

history = model.fit_generator(train_generator, epochs=25, validation_data = validation_generator, verbose = 1)

model.save("sp_recog.h5")

NameError: name 'validation_generator' is not defined